<a href="https://colab.research.google.com/github/ttury/dcinside_corpus_croller/blob/main/dc_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf
from transformer import *


train_data = pd.read_csv('./drive/MyDrive/data/dc_data_daego.csv')

print(len(train_data))
print(train_data.isnull().sum())
train_data.head()

445
Unnamed: 0     0
title          0
url            0
content       15
dtype: int64


,Unnamed: 0,title,url,content
0,0,1학년들 너무 무서워요,https://gall.dcinside.com/mini/board/view/?id=...,집에 오는데\n버스 정류장에 있는 여자보고 이년 저년 거리고\n토토 얘기 하고\n아...
1,1,수능 준비하느라 고생하는 3학년들,https://gall.dcinside.com/mini/board/view/?id=...,1년만 더 힘내세요! 아자아자!
2,2,Shangus,https://gall.dcinside.com/mini/board/view/?id=...,그립습니다..
3,3,대붕이들아;;,https://gall.dcinside.com/mini/board/view/?id=...,반에서 릴나스 보면 개추 ㅋㅋ\n일단 나부터\nhttps://youtu.be/UTH...
4,4,대고갤 실체,https://gall.dcinside.com/mini/board/view/?id=...,NaN


In [ ]:
# transformer.py 파일 업로드 필요

from google.colab import files

src = list(files.upload().values())[0]
open('transformer.py','wb').write(src)

Saving transformer.py to transformer.py


0

In [ ]:
questions = list()
for sentence in train_data['title']:
  sentence = re.sub(r"([?.!,])", r" \1 ", str(sentence))
  sentence = sentence.strip() # 양쪽 공백 제거
  questions.append(sentence)

answers = list()
for sentence in train_data['content']:
  sentence = re.sub(r"([?.!,])", r" \1 ", str(sentence))
  sentence = sentence.strip() # 양쪽 공백 제거
  answers.append(sentence)

print(questions[:5])
print(answers[:5])

['1학년들 너무 무서워요', '수능 준비하느라 고생하는 3학년들', 'Shangus', '대붕이들아;;', '대고갤 실체']
['집에 오는데\n버스 정류장에 있는 여자보고 이년 저년 거리고\n토토 얘기 하고\n아무개가 담배 2번 걸려서 사회 봉사를 갔다는 얘기도 하고\n화장실에서 담배도 피운다고 하고\n너무 무서워요 ㅠㅠ', '1년만 더 힘내세요 !  아자아자 !', '그립습니다 .  .', '반에서 릴나스\xa0보면 개추 ㅋㅋ\n일단 나부터\nhttps://youtu . be/UTHLKHL_whs', 'nan']


In [ ]:
# 서브워드텍스트인코더를 이용해 단어 사전 생성

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13
)

# 단어 사전에 인덱스 2개 추가(디코더의 입력에 사용하는 SOS, EOS 토큰)
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2

print('SOS 토큰 번호 :', START_TOKEN)
print('EOS 토큰 번호 :', END_TOKEN)
print('단어 집합의 크기 :', VOCAB_SIZE)
# 패딩 토큰 + 8179번 토큰 = 단어 집합의 크기

SOS 토큰 번호 : [5711]
EOS 토큰 번호 : [5712]
단어 집합의 크기 : 5713


In [ ]:
# 서브워드텍스트인코더 토크나이저의 encode()를 사용하여 정수 인코딩

sample_string = questions[21]

tokenized_string = tokenizer.encode(sample_string)
print('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

original_string = tokenizer.decode(tokenized_string)
print('기존 문장 : {}'.format(original_string))

for token in tokenized_string:
  print ('{} ----> {}'.format(token, tokenizer.decode([token])))
  # decode의 경우 리스트를 인자로 받는다
  # 한 어절이 정수 인코딩 후에는 여러 서브워드로 맵핑될 수 있음

print(len(tokenized_string))

정수 인코딩 후의 문장 : [199, 82, 3584, 20]
기존 문장 : 내일 내 몸에 .
199 ----> 내일 
82 ----> 내 
3584 ----> 몸에
20 ---->  .
4


In [ ]:
max_length = 0
for question in questions:
  tokenized_question = tokenizer.encode(question)
  max_length = max(max_length, len(tokenized_question))

print(max_length) # question : 10 / 시작, 끝 토큰 포함 x

max_length = 0
for answer in answers:
  tokenized_answer = tokenizer.encode(answer)
  max_length = max(max_length, len(tokenized_answer))
  
print(max_length) # question : 10 / 시작, 끝 토큰 포함 x

10
356


In [ ]:
MAX_LENGTH = 360

def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sent1, sent2) in zip(inputs, outputs):
    sent1 = START_TOKEN + tokenizer.encode(sent1) + END_TOKEN
    sent2 = START_TOKEN + tokenizer.encode(sent2) + END_TOKEN

    tokenized_inputs.append(sent1)
    tokenized_outputs.append(sent2)
  
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post') # 뒤를 패딩
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs

questions, answers = tokenize_and_filter(questions, answers)

In [ ]:
print('Q 데이터의 크기(shape) :', questions.shape)
print('A 데이터의 크기(shape) :', answers.shape)
print()
print(questions[0])
print(answers[0])

AttributeError: ignored

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs' : questions,
        'dec_inputs' : answers[:, :-1] # 디코더 입력, 마지막 패딩 토큰 제거
    },
    {
        'outputs' : answers[:, 1:] # 디코더 실제값(교사 강요), 시작 토큰 제거
    },
))

dataset = dataset.cache()
# 전처리 시간을 줄여줌
dataset = dataset.shuffle(BUFFER_SIZE)
# 데이터를 버퍼에 넣고 무작위로 빼내며 셔플, buffer_size가 데이터 크기보다 커야 함
dataset = dataset.batch(BATCH_SIZE)
# 데이터를 batch_size 개수대로 분리
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
# prefetch : 학습중일때, 데이터 로드시간을 줄이기 위해 메모리에 미리 적재시킴
# AUTOTUNE : 네트워크가 알아서 적재시킬 양을 설정해라

print(answers[0]) # 기존 샘플
print(answers[:1][:, :-1]) # 디코더 입력 샘플
print(answers[:1][:, 1:]) # 디코더 정답 샘플

[5711  330 2376 5465  389 1855   38 2418  350 1922 4727 5465 1386  245
  167 5465 2710 4110 5296  297  259 3298 4829 2532  167 5465  490 4108
 1286  167 5465   44 3554  146 5712    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [ ]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size = VOCAB_SIZE,
    num_layers = NUM_LAYERS,
    dff = DFF,
    d_model = D_MODEL,
    num_heads = NUM_HEADS,
    dropout = DROPOUT)

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y, y_hat): # metrics에 적용할 함수
  y = tf.reshape(y, shape=(-1, MAX_LENGTH - 1))
  # -1은 None과 같은 이유, 시작 토큰을 제거했으므로 길이 1 감소
  return tf.keras.metrics.sparse_categorical_accuracy(y, y_hat)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)

In [ ]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
  # 첫번째 차원 추가
  
  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    predictions = predictions[: :, -1:]
    # 문제 발생
    # 해결 : 학습 자료와 다르게 단어 벡터가 세번째 축에 위치했었음
    # 슬라이싱을 세번째 축으로 함으로서 concat 연산이 올바르게 수행되게 함
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    if tf.equal(predicted_id, END_TOKEN[0]):
      break
    
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [ ]:
def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [token for token in prediction if token < tokenizer.vocab_size])
  
  print('Input : {}'.format(sentence))
  print('Output : {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

In [ ]:
while(True):
  sentence = input()
  if sentence == "exit":
    break
  output = predict(sentence)

1학년들 너무 무서워요


In [ ]:
model.save_weights('./drive/MyDrive/models/DC_failed_1')

In [ ]:
 model.load_weights('./drive/MyDrive/models/DC_failed_1'